### Based on https://developer.hashicorp.com/vault/tutorials/kubernetes/kubernetes-minikube-tls

In [14]:
%%bash
minikube delete -p bbva

* Eliminando "bbva" en docker...
* Eliminando contenedor "bbva" ...
* Eliminando /Users/jose/.minikube/machines/bbva...
* Removed all traces of the "bbva" cluster.


In [15]:
%%bash
# Initialize docker first
#sleep 30
minikube start -p bbva

* [bbva] minikube v1.29.0 en Darwin 13.3.1 (arm64)
* Controlador docker seleccionado automáticamente. Otras opciones: virtualbox, ssh
* Using Docker Desktop driver with root privileges
* Starting control plane node bbva in cluster bbva
* Pulling base image ...
* Creando docker container (CPUs=2, Memory=4000MB) ...
* Preparando Kubernetes v1.26.1 en Docker 20.10.23...
  - Generando certificados y llaves
  - Iniciando plano de control
  - Configurando reglas RBAC...
* Configurando CNI bridge CNI ...
  - Using image gcr.io/k8s-minikube/storage-provisioner:v5
* Verifying Kubernetes components...
* Complementos habilitados: default-storageclass, storage-provisioner
* Done! kubectl is now configured to use "bbva" cluster and "default" namespace by default


In [16]:
%%bash
minikube status -p bbva
kubectl config use-context bbva

bbva
type: Control Plane
host: Running
kubelet: Running
apiserver: Running
kubeconfig: Configured

Switched to context "bbva".


In [17]:
%%bash
helm repo add hashicorp https://helm.releases.hashicorp.com
helm repo update

"hashicorp" already exists with the same configuration, skipping
Hang tight while we grab the latest from your chart repositories...
...Successfully got an update from the "jaeger-all-in-one" chart repository
...Successfully got an update from the "secrets-store-csi-driver" chart repository
...Successfully got an update from the "hashicorp" chart repository
...Successfully got an update from the "jaegertracing" chart repository
...Successfully got an update from the "datadog" chart repository
...Successfully got an update from the "jetstack" chart repository
...Successfully got an update from the "signoz" chart repository
...Successfully got an update from the "bitnami" chart repository
Update Complete. ⎈Happy Helming!⎈


In [18]:
%%bash
rm -rf /tmp/vault
mkdir /tmp/vault

In [30]:
%env WORKDIR=/tmp/vault
%env VAULT_K8S_NAMESPACE=vault
%env VAULT_HELM_RELEASE_NAME=vault
%env VAULT_SERVICE_NAME=vault-internal 
%env K8S_CLUSTER_NAME=cluster.local 

env: WORKDIR=/tmp/vault
env: VAULT_K8S_NAMESPACE=vault
env: VAULT_HELM_RELEASE_NAME=vault
env: VAULT_SERVICE_NAME=vault-internal
env: K8S_CLUSTER_NAME=cluster.local


### Adding enterprise license based on https://developer.hashicorp.com/vault/docs/platform/k8s/helm/enterprise

In [31]:
%%bash 
secret=$(cat /Users/jose/Demo/Vault_ADP/ADP_PlayGround/vault.hclic)
kubectl create secret generic vault-ent-license --from-literal="license=${secret}" -n $VAULT_K8S_NAMESPACE

secret/vault-ent-license created


In [32]:
%%bash
cat > ${WORKDIR}/overrides.yaml <<EOF
global:
   enabled: true
   tlsDisable: false # Disabling TLS to avoid issues when connecting to Vault via port forwarding

injector:
   enabled: true
   #repository: registry.connect.redhat.com/hashicorp/vault-k8s
   # Testing on Mac M1 so moving to arm image
   image:
      repository: docker.io/hashicorp/vault-k8s
      #repository: registry.connect.redhat.com/hashicorp/vault-k8s
      # Testing on Mac M1 so moving to arm image
   agentImage:
      repository: docker.io/hashicorp/vault
      #repository: registry.connect.redhat.com/hashicorp/vault
      # Testing on Mac M1 so moving to arm image

# Supported log levels include: trace, debug, info, warn, error
logLevel: "trace" # Set to trace for initial troubleshooting, info for normal operation

server:
# config.yaml
   image:
      repository: docker.io/hashicorp/vault-enterprise
      tag: 1.13.1-ent

   enterpriseLicense:
      secretName: vault-ent-license
   extraEnvironmentVars:
      VAULT_CACERT: /vault/userconfig/vault-ha-tls/vault.ca
      VAULT_TLSCERT: /vault/userconfig/vault-ha-tls/vault.crt
      VAULT_TLSKEY: /vault/userconfig/vault-ha-tls/vault.key
   volumes:
      - name: userconfig-vault-ha-tls
        secret:
         defaultMode: 420
         secretName: vault-ha-tls
   volumeMounts:
      - mountPath: /vault/userconfig/vault-ha-tls
        name: userconfig-vault-ha-tls
        readOnly: true
   standalone:
      enabled: false
   affinity: ""
   ha:
      enabled: true
      replicas: 3
      raft:
         enabled: true
         setNodeId: true
         config: |
            ui = true
            listener "tcp" {
               tls_disable = 0 # Disabling TLS to avoid issues when connecting to Vault via port forwarding
               address = "[::]:8200"
               cluster_address = "[::]:8201"
               tls_cert_file = "/vault/userconfig/vault-ha-tls/vault.crt"
               tls_key_file  = "/vault/userconfig/vault-ha-tls/vault.key"
               tls_client_ca_file = "/vault/userconfig/vault-ha-tls/vault.ca"
               api_address = "https://vault-active.vault.svc.cluster.local:8200"
            }
            storage "raft" {
               path = "/vault/data"
            
               retry_join {
                  auto_join             = "provider=k8s namespace=vault label_selector=\"component=server,app.kubernetes.io/name=vault\""
                  auto_join_scheme      = "https"
                  leader_ca_cert_file   = "/vault/userconfig/vault-ha-tls/vault.ca"
                  leader_tls_servername = "vaultpr-0.vault-internal" #Tiene que matchear una SAN del certificado
               }
            
            }
            disable_mlock = true
            service_registration "kubernetes" {}
EOF


In [ ]:
%%bash
helm install -n $VAULT_K8S_NAMESPACE $VAULT_HELM_RELEASE_NAME hashicorp/vault -f ${WORKDIR}/overrides.yaml

vault-agent-injector-5946fd9b8-fld4n   0/1     Pending            0             0s
vault-agent-injector-5946fd9b8-fld4n   0/1     Pending            0             0s
vault-agent-injector-5946fd9b8-fld4n   0/1     ContainerCreating   0             0s
NAME: vault
LAST DEPLOYED: Thu Apr 27 11:04:37 2023
NAMESPACE: vault
STATUS: deployed
REVISION: 1
NOTES:
Thank you for installing HashiCorp Vault!

Now that you have deployed Vault, you should look over the docs on using
Vault with Kubernetes available here:

https://www.vaultproject.io/docs/


Your release is named vault. To learn more about the release, try:

  $ helm status vault
  $ helm get manifest vault


vault-0                                0/1     Pending             0             0s
vault-0                                0/1     Pending             0             0s
vault-1                                0/1     Pending             0             0s
vault-1                                0/1     Pending             0             0s
vault-0                                0/1     ContainerCreating   0             0s
vault-2                                0/1     Pending             0             0s
vault-2                                0/1     Pending             0             0s
vault-agent-injector-5946fd9b8-fld4n   0/1     Running             0             1s
vault-0                                0/1     ContainerCreating   0             1s
vault-1                                0/1     Pending             0             1s
vault-1                                0/1     ContainerCreating   0             1s
vault-0                                0/1     Running             0        

In [ ]:
%%bash
kubectl -n $VAULT_K8S_NAMESPACE get pods --watch

NAME                                   READY   STATUS             RESTARTS     AGE
vault-0                                0/1     CrashLoopBackOff   1 (7s ago)   27s
vault-1                                0/1     CrashLoopBackOff   1 (8s ago)   26s
vault-2                                0/1     CrashLoopBackOff   1 (9s ago)   26s
vault-agent-injector-5946fd9b8-5nqvs   1/1     Running            0            57s
vault-2                                0/1     Error              2 (14s ago)   31s
vault-2                                0/1     CrashLoopBackOff   2 (2s ago)    33s
vault-0                                0/1     Error              2 (15s ago)   35s
vault-0                                0/1     CrashLoopBackOff   2 (1s ago)    36s
vault-1                                0/1     Error              2 (18s ago)   36s
vault-1                                0/1     CrashLoopBackOff   2 (3s ago)    39s
Error while terminating subprocess (pid=6675): 


vault-0                                0/1     Error              3 (25s ago)   60s
vault-0                                0/1     CrashLoopBackOff   3 (1s ago)    61s
vault-2                                0/1     Error              3 (31s ago)   62s
vault-2                                0/1     CrashLoopBackOff   3 (1s ago)    63s
vault-1                                0/1     Error              3 (32s ago)   68s
vault-1                                0/1     CrashLoopBackOff   3 (1s ago)    69s
vault-2                                0/1     Error              4 (43s ago)   105s
vault-2                                0/1     CrashLoopBackOff   4 (3s ago)    108s
vault-1                                0/1     Error              4 (45s ago)   113s
vault-0                                0/1     Error              4 (54s ago)   114s
vault-1                                0/1     CrashLoopBackOff   4 (1s ago)    114s
vault-0                                0/1     CrashLoopBackOff   4 (2s

In [34]:
%%bash
kubectl exec -n $VAULT_K8S_NAMESPACE vault-0 -- vault operator init \
    -key-shares=1 \
    -key-threshold=1 \
    -format=json > ${WORKDIR}/cluster-keys.json

vault-0                                0/1     Running             0             20s


In [35]:
%%bash
jq -r ".unseal_keys_b64[]" ${WORKDIR}/cluster-keys.json
VAULT_UNSEAL_KEY=$(jq -r ".unseal_keys_b64[]" ${WORKDIR}/cluster-keys.json)

jaExo/rK+6uFI3VzKHIdNmEDufaVY+kv3OBT9xQbVHE=


In [36]:
%%bash
kubectl exec -n $VAULT_K8S_NAMESPACE vault-0 -- vault operator unseal $(jq -r ".unseal_keys_b64[]" ${WORKDIR}/cluster-keys.json)

vault-0                                0/1     Running             0             28s
vault-0                                0/1     Running             0             28s
Key                     Value
---                     -----
Seal Type               shamir
Initialized             true
Sealed                  false
Total Shares            1
Threshold               1
Version                 1.13.1+ent
Build Date              2023-03-23T20:09:57Z
Storage Type            raft
Cluster Name            vault-cluster-b636cc4a
Cluster ID              c1a91f42-31d2-ba5b-e4e2-2e91c445a8ed
HA Enabled              true
HA Cluster              https://vault-0.vault-internal:8201
HA Mode                 active
Active Since            2023-04-27T09:05:07.219118672Z
Raft Committed Index    72
Raft Applied Index      71
Last WAL                29


vault-0                                1/1     Running             0             28s


In [37]:
%%bash
kubectl exec -n $VAULT_K8S_NAMESPACE vault-1 -- vault operator unseal $(jq -r ".unseal_keys_b64[]" ${WORKDIR}/cluster-keys.json)

Key                Value
---                -----
Seal Type          shamir
Initialized        true
Sealed             true
Total Shares       1
Threshold          1
Unseal Progress    0/1
Unseal Nonce       n/a
Version            1.13.1+ent
Build Date         2023-03-23T20:09:57Z
Storage Type       raft
HA Enabled         true


vault-1                                0/1     Running             0             33s
vault-1                                0/1     Running             0             33s
vault-1                                1/1     Running             0             33s
vault-1                                1/1     Running             0             36s


In [38]:
%%bash
kubectl exec -n $VAULT_K8S_NAMESPACE -ti vault-2 -- vault operator unseal $(jq -r ".unseal_keys_b64[]" ${WORKDIR}/cluster-keys.json)

Unable to use a TTY - input is not a terminal or the right kind of file


Key                Value
---                -----
Seal Type          shamir
Initialized        true
Sealed             true
Total Shares       1
Threshold          1
Unseal Progress    0/1
Unseal Nonce       n/a
Version            1.13.1+ent
Build Date         2023-03-23T20:09:57Z
Storage Type       raft
HA Enabled         true


vault-2                                0/1     Running             0             40s
vault-2                                0/1     Running             0             40s
vault-2                                1/1     Running             0             40s
vault-2                                1/1     Running             0             42s


In [39]:
%%bash
kubectl exec -n $VAULT_K8S_NAMESPACE -ti vault-0 -- vault status


Unable to use a TTY - input is not a terminal or the right kind of file


Key                     Value
---                     -----
Seal Type               shamir
Initialized             true
Sealed                  false
Total Shares            1
Threshold               1
Version                 1.13.1+ent
Build Date              2023-03-23T20:09:57Z
Storage Type            raft
Cluster Name            vault-cluster-b636cc4a
Cluster ID              c1a91f42-31d2-ba5b-e4e2-2e91c445a8ed
HA Enabled              true
HA Cluster              https://vault-0.vault-internal:8201
HA Mode                 active
Active Since            2023-04-27T09:05:07.219118672Z
Raft Committed Index    98
Raft Applied Index      98
Last WAL                32


In [40]:
%%bash
kubectl exec -n $VAULT_K8S_NAMESPACE -ti vault-0 --  vault license inspect


Unable to use a TTY - input is not a terminal or the right kind of file


Source: /vault/license/license (path from the VAULT_LICENSE_PATH environment variable)
Product: vault
License ID: 152295e1-3784-2641-bd01-bfdbc0a37c35
Customer ID: d9a6d7e8-e947-ea92-ec07-82c6901283c1
Installation ID: *
Issue Time: 2022-12-19 08:50:24.624461496 +0000 UTC
Start Time: 2022-12-19 00:00:00 +0000 UTC
Expiration Time: 2023-12-19 23:59:59.999 +0000 UTC
Termination Time: 2023-12-19 23:59:59.999 +0000 UTC
{"license_id":"152295e1-3784-2641-bd01-bfdbc0a37c35","customer_id":"d9a6d7e8-e947-ea92-ec07-82c6901283c1","installation_id":"*","issue_time":"2022-12-19T08:50:24.624461496Z","start_time":"2022-12-19T00:00:00Z","expiration_time":"2023-12-19T23:59:59.999Z","termination_time":"2023-12-19T23:59:59.999Z","flags":{"modules":["multi-dc-scale","governance-policy","advanced-data-protection-transform","advanced-data-protection-key-management"]},"features":["HSM","Performance Replication","DR Replication","MFA","Sentinel","Seal Wrapping","Control Groups","Performance Standby","Namespaces

In [ ]:
%%bash
kubectl exec -n $VAULT_K8S_NAMESPACE -ti vault-1 -- vault status

Unable to use a TTY - input is not a terminal or the right kind of file


Key                                    Value
---                                    -----
Seal Type                              shamir
Initialized                            true
Sealed                                 false
Total Shares                           1
Threshold                              1
Version                                1.13.1+ent
Build Date                             2023-03-23T20:09:57Z
Storage Type                           raft
Cluster Name                           vault-cluster-ff6949fa
Cluster ID                             b149ae66-90cc-472f-2c63-5d6051f553e8
HA Enabled                             true
HA Cluster                             https://vault-0.vault-internal:8201
HA Mode                                standby
Active Node Address                    https://10.244.0.4:8200
Performance Standby Node               true
Performance Standby Last Remote WAL    0
Raft Committed Index                   99
Raft Applied Index                     99


In [41]:
%%bash
kubectl exec -n $VAULT_K8S_NAMESPACE -ti vault-1 --  vault license inspect

Unable to use a TTY - input is not a terminal or the right kind of file


Source: /vault/license/license (path from the VAULT_LICENSE_PATH environment variable)
Product: vault
License ID: 152295e1-3784-2641-bd01-bfdbc0a37c35
Customer ID: d9a6d7e8-e947-ea92-ec07-82c6901283c1
Installation ID: *
Issue Time: 2022-12-19 08:50:24.624461496 +0000 UTC
Start Time: 2022-12-19 00:00:00 +0000 UTC
Expiration Time: 2023-12-19 23:59:59.999 +0000 UTC
Termination Time: 2023-12-19 23:59:59.999 +0000 UTC
{"license_id":"152295e1-3784-2641-bd01-bfdbc0a37c35","customer_id":"d9a6d7e8-e947-ea92-ec07-82c6901283c1","installation_id":"*","issue_time":"2022-12-19T08:50:24.624461496Z","start_time":"2022-12-19T00:00:00Z","expiration_time":"2023-12-19T23:59:59.999Z","termination_time":"2023-12-19T23:59:59.999Z","flags":{"modules":["multi-dc-scale","governance-policy","advanced-data-protection-transform","advanced-data-protection-key-management"]},"features":["HSM","Performance Replication","DR Replication","MFA","Sentinel","Seal Wrapping","Control Groups","Performance Standby","Namespaces

In [42]:
%%bash
kubectl exec -n $VAULT_K8S_NAMESPACE -ti vault-2 -- vault status

Unable to use a TTY - input is not a terminal or the right kind of file


Key                                    Value
---                                    -----
Seal Type                              shamir
Initialized                            true
Sealed                                 false
Total Shares                           1
Threshold                              1
Version                                1.13.1+ent
Build Date                             2023-03-23T20:09:57Z
Storage Type                           raft
Cluster Name                           vault-cluster-b636cc4a
Cluster ID                             c1a91f42-31d2-ba5b-e4e2-2e91c445a8ed
HA Enabled                             true
HA Cluster                             https://vault-0.vault-internal:8201
HA Mode                                standby
Active Node Address                    https://10.244.0.8:8200
Performance Standby Node               true
Performance Standby Last Remote WAL    0
Raft Committed Index                   110
Raft Applied Index                     110


In [43]:
%%bash
kubectl exec -n $VAULT_K8S_NAMESPACE -ti vault-2 --  vault license inspect

Unable to use a TTY - input is not a terminal or the right kind of file


Source: /vault/license/license (path from the VAULT_LICENSE_PATH environment variable)
Product: vault
License ID: 152295e1-3784-2641-bd01-bfdbc0a37c35
Customer ID: d9a6d7e8-e947-ea92-ec07-82c6901283c1
Installation ID: *
Issue Time: 2022-12-19 08:50:24.624461496 +0000 UTC
Start Time: 2022-12-19 00:00:00 +0000 UTC
Expiration Time: 2023-12-19 23:59:59.999 +0000 UTC
Termination Time: 2023-12-19 23:59:59.999 +0000 UTC
{"license_id":"152295e1-3784-2641-bd01-bfdbc0a37c35","customer_id":"d9a6d7e8-e947-ea92-ec07-82c6901283c1","installation_id":"*","issue_time":"2022-12-19T08:50:24.624461496Z","start_time":"2022-12-19T00:00:00Z","expiration_time":"2023-12-19T23:59:59.999Z","termination_time":"2023-12-19T23:59:59.999Z","flags":{"modules":["multi-dc-scale","governance-policy","advanced-data-protection-transform","advanced-data-protection-key-management"]},"features":["HSM","Performance Replication","DR Replication","MFA","Sentinel","Seal Wrapping","Control Groups","Performance Standby","Namespaces

## Get Root Token and port forward to Vault on port 8200

In [44]:
%%bash
cat ${WORKDIR}/cluster-keys.json | jq -r ".root_token"

hvs.22iw9eugNC3Kcdsv8Y9fHDHG


In [45]:
%%bash
#kubectl -n $VAULT_K8S_NAMESPACE get service vault
kubectl -n vault port-forward service/vault 8200:8200 

Forwarding from 127.0.0.1:8200 -> 8200
Forwarding from [::1]:8200 -> 8200
vault-agent-injector-5946fd9b8-fld4n   1/1     Terminating         0             104s
vault-agent-injector-5946fd9b8-fld4n   0/1     Terminating         0             105s
vault-agent-injector-5946fd9b8-fld4n   0/1     Terminating         0             105s
vault-agent-injector-5946fd9b8-fld4n   0/1     Terminating         0             105s
Handling connection for 8200
Handling connection for 8200
Handling connection for 8200
Handling connection for 8200
Handling connection for 8200
vault-secrets-operator-controller-manager-67cfc56bcd-n7c6d   0/2     Pending             0             0s
vault-secrets-operator-controller-manager-67cfc56bcd-n7c6d   0/2     Pending             0             0s
vault-secrets-operator-controller-manager-67cfc56bcd-n7c6d   0/2     ContainerCreating   0             0s
vault-secrets-operator-controller-manager-67cfc56bcd-n7c6d   1/2     Running             0             14s
vault-secret

E0427 14:07:22.479321    7187 portforward.go:233] lost connection to pod
